In [1]:
from IPython.html.services.config import ConfigManager
from IPython.utils.path import locate_profile
cm = ConfigManager(profile_dir=locate_profile(get_ipython().profile))
cm.update('livereveal', {
              'theme': 'sky',
              'transition': 'zoom',
              'start_slideshow_at': 'selected',
})

/home/ivan/miniconda2/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
/home/ivan/miniconda2/lib/python2.7/site-packages/IPython/utils/path.py:282: UserWarning: locate_profile has moved to the IPython.paths module
  warn("locate_profile has moved to the IPython.paths module")


{u'scroll': True,
 u'start_slideshow_at': 'selected',
 u'theme': 'sky',
 u'transition': 'zoom'}

# Lecture 6. Hierarchical matrices

## Previous lecture 
- Complexity analysis of the FMM 
- Connection between separability and low-rank approximation
- Algebraic interpretation of the Barnes-Hut method (H-matrices)
- Algebraic interpretation of the FMM method (H^2-matrices)

## Todays lecture
- Finalizing the FMM (multipole, local)
- Estimates for asymptotically smooth functions
- How we construct H-matrix approximation in a "black-box" way
- Maxvol algorithm and adaptive cross approximation

## Dense matrix-by-vector product

$$V_i = \sum_{j} K_{ij} q_j, \quad j = 1, \ldots, N, \quad i = 1, \ldots, M.$$

It happens when:

- $N$-body: $V_i = \sum_{j} q_j K(x_i, y_j)$.
- collocation: $V_i = \sum_{j} q_j \int_{supp v_j} K(x_i, y) v_j(y) dy$.
- Galerkin: $V_i = \sum_{j} q_j \int_{supp v_i} \int_{supp v_j} K(x, y) v_i(x) v_j(y) dx dy$.

Each $(i, j)$ is associated as a pair of points $(x_i, y_j)$ in $\mathbb{R}^d, d = 1, 2, 3$.


## General scheme 

Given the set of sources $x_i, \quad i = 1, \ldots N$ and the set of receivers $y_j, \quad  j = 1, \ldots, M$. 

- Construct a cluster tree for sources with nodes $s$
- Construct a cluster tree for receivers with nodes $t$
- Based on the separability criteria between nodes $(t, s)$ create **interaction list**,
  which is a set of pairs $(t, s)$. The number of such pairs is $\mathcal{O}(N+M)$.

## Main requirement

The main requirement on the "far zone" and the "Kernel" is that

If $y$ is closer to $z$, rather than to $x$, then

$$K(x, y) \approx K'_N(x, y) \approx \sum_{l=0}^N \sum_{m \in M_l} t_{lm}(y, z) T_{lm}(x, z),$$

and the error bound looks like

$$\Vert K(x, y) - K'(x, y) \Vert \leq c_N \left( \frac{\Vert y - z \Vert}{\Vert x - z \Vert}\right)^N.$$

On the other hand, if $y$ is closer to $x$ rather than $z$ a **dual decomposition** holds.


$$K(x, y) \approx K''_N(x, y)  \sum_{l=0}^N \sum_{m \in M_l} t_{lm}(x, z) T_{lm}(y, z),$$

with a bound
$$\Vert K(x, y) - K''(x, y) \Vert \leq c_N \left( \frac{\Vert x - z \Vert}{\Vert y - z \Vert}\right)^N.$$


## Far field

The **far field** of the set $Y$ with a center $z \in Y$ 
$\mathcal{F}_q(Y, z)$ is defined as

$$\mathcal{F}_q(Y, z) = \{x: \Vert x - z \Vert \leq  \frac{\Vert y - z \Vert}{q}, \forall y \in Y \}$$

**Main lemma**

If $y \in Y$ and $x \in \mathcal{F}_q(Y, z)$ then

$$|K(x, y) - K'_N(x,y)| \leq c_N q^N.$$ 

## Far field approximation
In the far field $x \in \mathcal{F}_q(Y, z)$ 
the field of point charges

$$f(x) = \sum_j q_j K(x, y_j)$$ 
is approximated by the **multipole expansion**

$$M(x) = \sum_{l=0}^N \sum_{m \in M_l} M_{lm} T_{lm}(x, z),$$

where 

$$
  M_{lm} = \sum_{j} t_{lm}(y_j, z) q_j.
$$

## Local expansion

If in the point $x \in Y$ the field from charges $y_j \in \mathcal{F}_q(Y, z)$ in the **far field** is calculated,

it is then approximated by the **local expansion**

$$L(x) = \sum_{l=0}^N \sum_{m \in \mathcal{M}_l} L_{lm} t_{lm}(x, z), \quad  L_{lm} = \sum_j  T_{lm}(y_j, z) q_j$$

## Examples of local & multipole expansion

For the specific kernel, one can write down the specific expansions (you need two).
Introduce spherical coordinates with a center in $z$. The point $x$ has coordinates $(r_x, \phi_x, \theta_x)$, 
the point $y$ has coordinates $(r_y, \phi_y, \theta_y)$.

$$K(x, y) = \frac{1}{\Vert x - y \Vert}.$$

Then, $$M(x) = \sum_{l=0}^N \sum_{m=-l}^{l} M_{lm} \frac{1}{r^{l+1}_x} Y^m_l(\phi_x, \theta_x), 
\quad L(x) = \sum_{l=0}^N \sum_{m=-l}^l L_{lm} r^l_x Y^m_{l}(\phi_x, \theta_x),$$

where $Y^m_l$ are **spherical harmonics.**


## Helmholtz Kernel

For the Helmholtz kernel $K(x, y) = \frac{e^{ikr}}{r}, \quad r = \Vert x - y \Vert,$
the expansions take the form

$$M(x) = \sum_{l=0}^N \sum_{m=-l}^l M_{lm} H^{(1)}_l(k r_x) Y^m_l(\theta_x, \phi_x), \quad 
 L(x) = \sum_{l=0}^N \sum_{m=-l}^l L_{lm} J^{(1)}_l(k r_x) Y^{m}_l(\theta_x, \phi_x),$$
 where $H^{(1)}$ and $J^{(1)}$ are **Hankel** and **Bessel** functions of the first kind.


## Computing the bounds

The class of **asymptotically smooth** functions. The function $f$ of many variables is said to be **asymptotically smooth** if 

$$\Vert \partial^{p} f(x, y) \Vert \leq c_p \Vert x - y \Vert^{(g-p)},$$

with a "reasonable" bound on $c_p$: 
    $$c_p \leq c d^p p!.$$
    For $f(x, y) = \frac{1}{\Vert x - y \Vert}$ we have $g = 1$ and $c_p \leq 4^p p!$ (more accurate estimates give $c_p \leq 2^p$).

## Using the bounds

The main result is the **rank bound** for matrices generated by asymptotically smooth functions.

For any two separated sets of sources and receivers such that  $y_j$ are located inside a cube

$$C = \{y: \Vert y - \eta \Vert_{\infty} \leq a/2 \}.$$

and $x_i$ lies inside the set

$$\{x : \Vert x - y \Vert_{\infty}  \geq \sigma a, \quad \forall y \in C \}, $$

then the interaction matrix 

$$A = [A_{ij}], \quad A_{ij} = K(x_i, y_j)$$

can be written as 

$$A = T_p + R_p, $$

where

$$\mathrm{rank}(R_p) \leq p^m, \quad \Vert R_p \Vert^2_F \leq n_x n_y c^2 (\sigma a)^{2g} \left(\frac{dm}{2 \sigma}\right)^{2p}.$$

## What does it mean

$$A = T_p + R_p, $$

where

$$\mathrm{rank}(R_p) \leq p^m, \quad \Vert T_p \Vert^2_F \leq n_x n_y c^2 (\sigma a)^{2g} \left(\frac{dm}{2 \sigma}\right)^{2p}.$$

It means, that if 

$$\frac{dm}{2 \sigma} < 1 $$

(i.e., the cubes are sufficiently separated), the matrix can be approximated by a **low-rank matrix.**

This is a **general result** which holds for a class of matrices.

## Numerical algorithm

The partioning of the matrix can be done using **geometric information** about the sources and receivers.

But who do we compute the approximation?

<img src="pic/h-matrix.jpg" width=70%>

## H-matrices

In the hiearchical matrices we compute the **low-rank approximation** independently.

The problem is that the largest block has size $\mathcal{O}(N) \times \mathcal{O}(N)$.

The best rank-$r$ approximation can be computed by SVD,

but the complexity of the SVD is $\mathcal{O}(N^3)$.

Can we do better than SVD?


## Remembering the NLA: Skeleton decomposition

Even the computation of all the entries of a $N \times N$ matrix takes $\mathcal{O}(N^2)$ operations.

We do not want that.

Instead, we have an apriori knowledge of the fact that  the matrix is (approximately) **low-rank**.

The skeleton decomposition allows to recover a rank-$r$ entries for $r$ columns and $r$ rows.

<img src='pic/cross-pic.png' width=80% /img>

## Skeleton decomposition

The skeleton decomposition has the form

$$A = C\widehat{A}^{-1} R,$$

where $C$ is some columns of $A$, $R$ are some rows of $A$ and $\widehat{A}$ is the submatrix on the intersection.

It means, that rank-$r$ matrix can be exactly recovered from $r$ columns and $r$ rows, and only $2Nr$ elements need to be computed.

## How to select a good submatrix

In practice, if the submatrix is **bad**, the approximation can be **bad**.n{

For example, for a matrix

$$
   A = \begin{bmatrix}
   1 & 1 & 2 \\
   1 & 1.001 & 3  \\
   1 & 1    & 4 \\
   \end{bmatrix}
$$

The submatrix located in first two rows and two columns is really bad for rank-$2$ approximation.

## How to select a good submatrix(2)

The submatrix selection is an **art**: for any strategy you can design  a matrix that the strategy will fail.

In practice, however, the matrices come from a certain class, where the submatrix selection is possible.


The **maxvol** principle (Goreinov & Tyrtyshnikov) gives the **existence** result for such a submatrix:

If $\widehat{A}$ has the largest volume (the volume of the matrix is defined as an absolute value of the determinant) 

Then

$$\Vert A - A_{skel} \Vert \leq (r + 1) \sigma_{r+1}, $$

where $\sigma_{r+1}$ is the $(r+1)$-th singular value of the matrix.

## Maxvol-iteration

Then the simplest adaptive strategy for column/row selection is the **alternating optimization**.

We take $r$ columns from the matrix, compute them, 

and find the **maximum volume** in those columns; 

Compute the new rows, find the maximal volume and so on.

In this iteration the **volume** never decreases, thus the quality of the approximation.

## Illustration
 
As an  illustration, consider the case $r = 1$.

Then we need to find maximal absolute value, actually we need to find something that is not small.

Then we do **block coordinate descent**

$$i_* = \arg \max_i A_{ij_*}, \quad j_* = \arg \max_j A_{i_* j}$$

For a general rank-$r$ case we blocks of rows and columns.

## Final remark

But how to find a maximal-volume submatrix in a tall matrix?. 

Let $A$ be $n \times r$ matrix.
m
The total number of $r \times r$ submatrices is $\mathcal{O}(n^r)$, 
which is clearly intractable for $r > 1$.

## Maxvol algorithm: initialization 

The maxvol algorithm is a **greedy optimization** algorithm for searching for a good submatrix in a tall matrix.

Let $A$ be $n \times r$, $n \gg r$.

**Initialization:** We find some "sufficiently good" submatrix $\widehat{A}$ in $A$. We can do it by Gaussian elimination with column pivoting (please check).

## Maxvol algorithm: iteration step

The iteration step is as follows.

We order the rows of the matrix in such a way that

$$
A = P\begin{bmatrix}
\widehat{A} \\
B 
\end{bmatrix},$$ where

$P$ is the permutation matrix (it permutes the rows of the matrix).

Then, 

$$A \widehat{A}^{-1} = P \begin{bmatrix} I \\
Z \end{bmatrix}.
$$

Note that the ratio of the determinants did not change, so the position of the **maximum volume submatrix is the same**


## Maxvol algorithm: crucial step
We need to find the **maxvol algorithm** in the matrix
$$A \widehat{A}^{-1} = P \begin{bmatrix} I \\
Z \end{bmatrix}.
$$

We use the **greedy approach** by increasing the determinant by **permuting two rows** at a time.

The algebraic structures helps us to efficiently find such permutation at **very cost.**

Find 
$$
(i_*, j_) = \arg \max_{i, j} |Z_{ij}|.$$

Swap the $j$-th row and the $i$-th row.

Then the matrix will be

$$A \widehat{A}^{-1} = P' \begin{bmatrix} \begin{bmatrix}1 & \ldots \\
                                          \ldots & z & \ldots \\ 
                                          \ldots & \ldots & 1 
                                          \end{bmatrix}
                                          \\
                                          Z' 
                                          \end{bmatrix},$$
                                          
i.e. the top submatrix differs only in one row from the identity matrix.

## The result of the update

As the result of the update, 

we have

$$\det Z_{new} = \det (I - e_j e^{\top}_j + z_i e^{\top}_j) = \det(I + (z_i - e_j) e^{\top}_j) = 1 + (e_j, z_i - e_j) = 1  + z_{ij} - 1 = z_{ij}.$$

Thus, if $|Z_{ij}| > 1$ the new determinant is greater.

## Final scheme of the algorithm

1. Initialize the submatrix
2. Permute rows to make this submatrix the first one
3. multiply by the inverse of $\widehat{A}^{-1}$ from the right ($\mathcal{O}(nr^2 + r^3)$ operations)
4. Find $(i_*, j_*) = \arg \max_{ij} |Z_{ij}|$
5. If $|Z_{i_*, j_*}| < 1 + \delta$, stop
7. Otherwise, permute $i_*$, and $j_*$ rows and go to Step 3.

The final complexity is $\mathcal{O}(Inr^2)$ operations. We can make it $\mathcal{O}(I nr + nr^2)$, where $I$ is the number of iterations.

## Maxvol algorithm: efficient implementation

The efficient implementation is based on the inversion of the matrix after permuting two rows.

In fact, we only need to recompute $A \widehat{A}^{-1}$.

Note that 
$$A_{new} = A + e_i (e_j - e_i)^{\top} A + e_j (e_i - e^{\top}_j) A,$$

and

$$\widehat{A}_{new} = \widehat{A} + e_i (e_j - e_i)^{\top} A.$$

Using the Sherman-Woodbury-Morrison formula for the inverse of the rank-$1$ update of the matrix

we get

$$\widehat{A}^{-1}_{new} =  \widehat{A}^{-1} - \beta \widehat{A}^{-1} e_i (e_j - e_i)^{\top} A \widehat{A}^{-1},$$

where

$$\beta = \frac{1}{1 + (A\widehat{A} e_i, e_j - e_i)}.$$

The final formulas come by multiplying 

$A_{new}$ by $\widehat{A}_{new}^{-1}$ 

The total complexity of such update is $\mathcal{O}(nr)$.


## Maxvol algorithm: final remarks
- Typical number of iterations is 10
- QR decomposition can help the stability 
- This is an extremely important algorithm that has been known in other areas for **decades**
- The maximum-volume submatrix in a "tall" matrix gives us good interpolation points, 
  thus can be used for **optimal experiment design.**

## Alternating-maxvol summary

The summary of the algorithm:

Select $\mathcal{I}$ at random

$$
   J = \mathrm{maxvol}(A[I, :]^{\top})
$$

Then

$$ I = \mathrm{maxvol}(A[:, J]). $$

In this case, the volume never decreases, and $2-3$ iterations are often needed to find the right matrix.


## Adapting the rank

The problem of the alternating Maxvol algorithm is that the number of columns (approximate rank) has to be fixed.

It is typically not known in advance.

Thus, **greedy** algorithms have been developed, based on the idea of **cross approximation**

## Cross approximation and rank reduction

Cross approximation is just rank-$1$ skeleton decomposition.

Take column with index $j_*$, row with index $i_*$ and compute


$$A'_{ij} = A_{ij} - \frac{A_{ij_*} A_{i_* j}}{A_{i_*j_*}}.$$

The only requirement is that $A_{i_*, j_*}$ is not equal to $0$.

Then, the matrix $A'$ coincides with $A$ on the **cross** comprised from the $i_*$ row and $j_*$ column.

## Cross approximation and rank reduction(2)

The subtraction of the cross leads to the rank reduction:

$$\mathrm{rank}(A') = \mathrm{rank}(A) - 1,$$

i.e. if the matrix $A$ had rank $1$, after $r$ such steps we will have zero matrix.

Moreover, each cross is a **rank-1** matrix, thus

$$A_r = 0 = A_{r-1} - u_{r-1} v^{\top}_{r-1} = \ldots = A - \sum_{k=1}^r u_k v^{\top}_k, $$

thus

$$A = \sum_{k=1}^r u_k v^{\top}_k,$$

i.e. the required low-rank approximation.

Also, the cross of $A_k$ at the $k$-th can be computed cheaply, since

$$A_k = A - UV^{\top},$$

where $U$ and $V$ have $k$ columns, 

and 

$$A_k e_j = A e_j - U V^{\top} e_j$$ can be computed by computing the column of $A$ and $nk$ additional operations.

## Selection of pivots

Selection of pivots $(i_*, j_*)$ is the crucial point; 

The larger is the modulus, the better is the approximation, 

i.e. we have to find large values in the remainder.

One of the ways is to do **partial pivoting**.

If you do row pivoting (i.e. fix $i_*$ and find $j_*$ as the maximal element in a row)

the cross can be computed in a stable way since

$$\left|\frac{a_{ij_*}}{a_{i_* j_*}}\right| \leq 1, $$

and it is not difficult to see that the error will grow at most by a factor of $2$ at each step.

## More robust pivoting

More robust pivoting come from the Gaussian elimination:

- Rook pivoting (coordinate descent)
- Active submatrices (basically, redoing maxvol after $k$ steps)

The main reason why such methods work is that being shown $r$ elements from the row we are able to determine,

which are mostly linearly independent.

## CA and Wedderburn

One step of CA can be written as a rank-1 update of the matrix:

$$A_{k+1} = A_k - \frac{A e_i e^{\top}_j A}{(A e_i, e_j)}.$$

This is an example of a more general class of Wedderburn updates of the matrix:

$$A_{k+1} = A_k - \frac{A u v^{\top} A}{(A u, v)}.$$

For any pair of $(u, v)$ they reduce the rank, and the best "pivot" corresponds to the 
maximum of $|(Au, v)|$ which is given by the singular vector corresponding to the largest singular value.

Moreover, any matrix decomposition (LU, QR) can be obtained from this formula.

## Note on the theory

There is not too much theory on why these methods work.

For any strategy, there are examples, where they will not work.

For a class of function-related matrices there is a result by Demanet et. al

who showed that if the matrix $A$ can be written as

$$A = U \Phi V^{\top} + E,$$

and $U^{\top} U = V^{\top} V = I_r$

and $U$ and $V$ satisfy **incoherence properties**

$$\max |U_{ij}| \leq \frac{\mu}{\sqrt{N}}, \quad \max |V_{ij}| \leq \frac{\mu}{\sqrt{M}},$$

(it means, that there are no **gaps** in singular values) then it is sufficient to pick any 

$$p = r \log n$$ columns.

Still a lot to be done!

## Summary 
Today we talked about:

- A little bit on FMM from algebra part & estimates
- Adaptive cross approximation & maxvol algorithm

## Next lecture
- Working with H- and H-2 matrices: addition, multiplication, inversion
- Connection to sparse matrices

In [4]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()